# Modularized Advection of a Cosine Wave

Let's revisit the advection of a cosine wave, but this time modularize the different elements of the code we wrote to perform the predicted values of our wave function. Recall our function at $t=0$, our initial condition, is given by

$$ \psi(x,t=0) = \cos \Big( \frac{2 \pi x}{1000 \text{ m}} \Big) .$$

Our advection differential equation amidst a constant background flow $u$ is 

$$ \displaystyle \frac{\partial \psi}{\partial t} = - u \frac{\partial \psi}{\partial x} $$

which we use finite differencing, forward in time and backward in space, to get future values of $\psi$ via 

$$ \psi^{(n+1)} = \psi^{(n)} - u \Delta t \frac{\psi_{i} - \psi_{i-1}}{\Delta x} .$$

Our boundary condition is a left-periodic condition:

$$ \psi^{(n+1)}_0 = \psi^{(n+1)}_{n_x-1} .$$

Let's now implement the steps to predict future values of $\psi$ as before, but via modularization. Ultimately, this will help us add more and more parts to the code as we grow the complexity of the model.

## Import Modules

First, let's import the modules we need.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Create Grids

Now we need to create the grid for $x$ and the array of times $t$.

In [ ]:
def create_grid(nx,dx,nt,dt):
    
    '''Create a grid for x and t, each starting at 0, defined by grid spacings and number of grid points'''
    
    # Generate the arrays
    x = np.arange(0,nx*dx,dx)
    t = np.arange(0,(nt+1)*dt,dt)
    
    return x,t

## Initial Conditions

Next, we establish a function to set the initial condition, $\psi(x,t=0)$.

In [ ]:
def initial_conditions(x,L):
    
    '''Define the initial conditions for the wave'''
    
    # Create the initial wave function
    psi = np.cos(2.*np.pi*x/L)
    
    return psi

## Copy Wave Functions

We also a function to copy one function to another. We do this primarily so that we do not have to store $n_t$ versions of $\psi$ but instead, only the $(n+1)$th and $n$th copies. 

In [ ]:
def copy_functions(psi1):
    
    '''Copy one function to another'''
        
    # Copy psi1 to psi2
    psi2 = np.copy(psi1)
    
    return psi2

## Advection

Now we update the new value of $\psi$ with the advection function.

In [ ]:
def advection(psi_future,psi_present,u,dt,nx,dx):

    '''Update the wave function via the advection equation'''

    # Update according to advection equation
    psi_future[1:nx] = psi_present[1:nx] - (u*dt*(psi_present[1:nx]-psi_present[0:nx-1])/dx)
    
    return psi_future

## Boundary Condition

Let's add a boundary condition function now.

In [ ]:
def boundary_condition(psi,nx):
    
    '''Apply a periodic boundary condition'''
    
    # Apply the periodic boundary condition to the left side
    psi[0] = psi[nx-1]
    
    return psi

## Plotting the Function

Finally, we can make a function to plot the wave for each time step.

In [ ]:
def plot_function(x,psi,time):
    
    '''Plot the function for time t'''

    # Plot function
    fig,ax = plt.subplots(figsize=(12,6));
    ax.plot(x,psi_future);
    ax.set_xlabel("$x$",fontsize=14);
    ax.set_ylabel("$\psi$",fontsize=14);
    ax.set_title(f"t = {t}",fontsize=18);
    ax.set_xlim(min(x),max(x))
    
def plot_hovmoller(x,t,psi_history):
    
    '''Plot the function in the x-t space'''
    
    # Plot a contour plot of the hovmoller
    fig,ax = plt.subplots(figsize=(12,6))
    contourf = ax.contourf(x,t,np.stack(psi_history,axis=0))
    ax.tick_params(right=True, top=True, labelright=True, labeltop=True);
    ax.set_xlabel("$x$",fontsize=14);
    ax.set_ylabel("$t$",fontsize=14);
    ax.set_title(f"$\psi$",fontsize=18);
    fig.colorbar(contourf);

## Putting It All Together

So now, we can call each of these functions in the proper order.

In [ ]:
# Define the model parameters
nx = 21
nt = 10
dx = 100.
dt = 10.
u = 10.
L = 1000.

# Generate the grid
x,t = create_grid(nx,dx,nt,dt)

# Set the initial conditions
psi_future = initial_conditions(x,L)

# Create a list that will store all updates
psi_history = [psi_future]

# Loop through times and update the wave function
for n in range(1,nt+1):
    
    # Advance n to n+1
    psi_present = copy_functions(psi_future)
        
    # Update with advection
    psi_future = advection(psi_future,psi_present,u,dt,nx,dx)
    
    # Apply boundary conditions
    psi_future = boundary_condition(psi_future,nx)
        
    # Build history of wave values to plot
    psi_history.append(np.copy(psi_future))
    
# Plot the final result
plot_hovmoller(x,t,psi_history)

## Using a Class Structure

More pythonically, we can wrap all of this into a class instead. Let's give that a try.

In [ ]:
class Advection_Of_Cosine_Wave:
    
    def __init__(self,nx,dx,nt,dt,u,L):
        self.nx = nx
        self.dx = dx
        self.nt = nt
        self.dt = dt
        self.u = u
        self.L = L
        
    def create_grid(self):
    
        '''Create a grid for x and t, each starting at 0, defined by grid spacings and number of grid points'''

        # Generate the arrays
        self.x = np.arange(0,self.nx*self.dx,self.dx)
        self.t = np.arange(0,(self.nt+1)*self.dt,self.dt)
        
    def initial_conditions(self):
    
        '''Define the initial conditions for the wave'''
    
        # Create the initial wave function
        self.psi_future = np.cos(2.*np.pi*self.x/self.L)
        
    def copy_psi(self):
    
        '''Advance to the next n and update psi_present'''
        
        # Copy the functions
        self.psi_present = np.copy(self.psi_future)
    
    def advection(self):

        '''Update the wave function via the advection equation'''

        # Update according to advection equation
        self.psi_future[1:self.nx] = self.psi_present[1:self.nx] - \
                                     (self.u*self.dt*(self.psi_present[1:self.nx]-self.psi_present[0:self.nx-1])/self.dx)
        
    def boundary_condition(self):
    
        '''Apply a periodic boundary condition'''
    
        # Apply the periodic boundary condition to the left side
        self.psi_future[0] = self.psi_future[self.nx-1]
    
    def plot_function(self,n):
    
        '''Plot the function for time t'''

        # Plot function
        fig,ax = plt.subplots(figsize=(12,6));
        ax.plot(self.x,self.psi_future);
        ax.set_xlabel("$x$",fontsize=14);
        ax.set_ylabel("$\psi$",fontsize=14);
        ax.set_title(f"t = {self.t[n]}",fontsize=18);
        ax.set_xlim(min(self.x),max(self.x))
    
    def plot_hovmoller(self,psi_history):

        '''Plot the function in the x-t space'''

        fig,ax = plt.subplots(figsize=(12,6))
        contourf = ax.contourf(self.x,self.t,np.stack(psi_history,axis=0))
        ax.tick_params(right=True, top=True, labelright=True, labeltop=True);
        ax.set_xlabel("$x$",fontsize=14);
        ax.set_ylabel("$t$",fontsize=14);
        ax.set_title(f"$\psi$",fontsize=14);
        fig.colorbar(contourf);
        
    def run_advection_model(self,plot=False,hovmoller=True):
    
        '''Initiate the functions above and loop through times and update the wave function'''
        
        # Create the grid and set the initial conditions
        wave.create_grid()
        wave.initial_conditions()
        
        # Create the history list for the wave 
        psi_history = [np.copy(wave.psi_future)]
        
        # Plot the initial condition
        if plot : wave.plot_function(0)

        # Loop through times and update the wave function
        for n in range(1,nt+1):
    
            # Advance n to n+1
            wave.copy_psi()

            # Update with advection
            wave.advection()

            # Apply boundary conditions
            wave.boundary_condition()

            # Plot the wave for this time step
            if plot : wave.plot_function(n)

            # Build history of wave values to plot
            psi_history.append(np.copy(wave.psi_future))
    
        # Plot the hovmoler diagram
        if hovmoller : wave.plot_hovmoller(psi_history)

And now we need only create an instance of the class with the parameters set and perform the driver function `run_advection_model`.

In [ ]:
wave = Advection_Of_Cosine_Wave(nx=21,nt=10,dx=100.,dt=10.,u=10.,L=1000.)
wave.run_advection_model()